**Requirements** 
* According to this [paper](https://arxiv.org/pdf/1904.01561.pdf), features are computed with [descriptastorus](https://github.com/bp-kelley/descriptastorus) package
* Install via: `pip install git+https://github.com/bp-kelley/descriptastorus`

## General imports

In [1]:
import sys 
sys.path.insert(0, "/")  # this depends on the notebook depth and must be adapted per notebook
# from compert.paths import DATA_DIR, EMBEDDING_DIR
sys.path.append("/root/project/src/project_src/state_transition_model/benchmark/chemCPA/chemCPA-main/chemCPA")
from paths import DATA_DIR, EMBEDDING_DIR

In [2]:
import numpy as np
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

## Load Smiles list

In [4]:
import pandas as pd 
import scanpy as sc
adata = sc.read("/root/project/data/preprocessed_data/state_transition_model_benchmark/chemCPA/lincs_preprocess_2.h5ad", backed=True)
unique_smiles = np.unique(adata.obs["canonical_smiles"])


In [5]:
smiles_df = pd.DataFrame(unique_smiles)
smiles_df.columns = ["smiles"]

In [6]:
smiles_list = smiles_df['smiles'].values

In [7]:
print(f'Number of smiles strings: {len(smiles_list)}')

Number of smiles strings: 7963


In [8]:
from descriptastorus.descriptors.DescriptorGenerator import MakeGenerator
generator = MakeGenerator(("RDKit2D",))
for name, numpy_type in generator.GetColumns():
    print(f"{name}({numpy_type.__name__})")

RDKit2D_calculated(bool)
BalabanJ(float64)
BertzCT(float64)
Chi0(float64)
Chi0n(float64)
Chi0v(float64)
Chi1(float64)
Chi1n(float64)
Chi1v(float64)
Chi2n(float64)
Chi2v(float64)
Chi3n(float64)
Chi3v(float64)
Chi4n(float64)
Chi4v(float64)
EState_VSA1(float64)
EState_VSA10(float64)
EState_VSA11(float64)
EState_VSA2(float64)
EState_VSA3(float64)
EState_VSA4(float64)
EState_VSA5(float64)
EState_VSA6(float64)
EState_VSA7(float64)
EState_VSA8(float64)
EState_VSA9(float64)
ExactMolWt(float64)
FpDensityMorgan1(float64)
FpDensityMorgan2(float64)
FpDensityMorgan3(float64)
FractionCSP3(float64)
HallKierAlpha(float64)
HeavyAtomCount(float64)
HeavyAtomMolWt(float64)
Ipc(float64)
Kappa1(float64)
Kappa2(float64)
Kappa3(float64)
LabuteASA(float64)
MaxAbsEStateIndex(float64)
MaxAbsPartialCharge(float64)
MaxEStateIndex(float64)
MaxPartialCharge(float64)
MinAbsEStateIndex(float64)
MinAbsPartialCharge(float64)
MinEStateIndex(float64)
MinPartialCharge(float64)
MolLogP(float64)
MolMR(float64)
MolWt(float64)

In [9]:
n_jobs = 1
data = Parallel(n_jobs=n_jobs)(delayed(generator.process)(smiles) for smiles in tqdm(smiles_list, position=0, leave=True) )

  0%|          | 0/7963 [00:00<?, ?it/s]

In [10]:
embedding = np.array(data)
embedding.shape

(7963, 201)

## Check `nans` and `infs`

Check for `nans`

In [11]:
drug_idx, feature_idx = np.where(np.isnan(embedding))
print(f'drug_idx:\n {drug_idx}')
print(f'feature_idx:\n {feature_idx}')

drug_idx:
 [1372 1372 1372 1372 1862 1862 1862 1862 1919 1919 1919 1919 3911 3911
 3911 3911 5927 5927 5927 5927 5928 5928 5928 5928 5929 5929 5929 5929
 6300 6300 6300 6300 7442 7442 7442 7442 7557 7557 7557 7557 7612 7612
 7612 7612]
feature_idx:
 [40 42 44 46 40 42 44 46 40 42 44 46 40 42 44 46 40 42 44 46 40 42 44 46
 40 42 44 46 40 42 44 46 40 42 44 46 40 42 44 46 40 42 44 46]


Check for `infs` and add to idx lists

In [12]:
drug_idx_infs, feature_idx_infs = np.where(np.isinf(embedding))

drug_idx = np.concatenate((drug_idx, drug_idx_infs))
feature_idx = np.concatenate((feature_idx, feature_idx_infs))

Features that have these invalid values:

In [13]:
np.array(generator.GetColumns())[np.unique(feature_idx)]

array([['MaxAbsPartialCharge', <class 'numpy.float64'>],
       ['MaxPartialCharge', <class 'numpy.float64'>],
       ['MinAbsPartialCharge', <class 'numpy.float64'>],
       ['MinPartialCharge', <class 'numpy.float64'>]], dtype=object)

Set values to `0`

In [14]:
embedding[drug_idx, feature_idx] 

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, inf, inf, inf, inf, inf, inf])

In [15]:
embedding[drug_idx, feature_idx] = 0

## Save

In [16]:
import pandas as pd

df = pd.DataFrame(data=embedding,index=smiles_list,columns=[f'latent_{i}' for i in range(embedding.shape[1])]) 

# Drop first feature from generator (RDKit2D_calculated)
df.drop(columns=['latent_0'], inplace=True)

# Drop columns with 0 standard deviation
threshold = 0.01
columns=[f'latent_{idx+1}' for idx in np.where(df.std() <= threshold)[0]]
print(f'Deleting columns with std<={threshold}: {columns}')
df.drop(columns=[f'latent_{idx+1}' for idx in np.where(df.std() <= 0.01)[0]], inplace=True)

Deleting columns with std<=0.01: ['latent_90', 'latent_103', 'latent_152', 'latent_164', 'latent_187', 'latent_196']


Check that correct columns were deleted: 

In [17]:
np.where(df.std() <= threshold)

(array([], dtype=int64),)

### Normalise dataframe

In [18]:
normalized_df=(df-df.mean())/df.std()

In [19]:
normalized_df.head()

,latent_1,latent_2,latent_3,latent_4,latent_5,latent_6,latent_7,latent_8,latent_9,latent_10,...,latent_190,latent_191,latent_192,latent_193,latent_194,latent_195,latent_197,latent_198,latent_199,latent_200
BrC1C(Br)C(Br)C(Br)C(Br)C1Br,1.828414,-1.981056,-1.401597,-1.634367,-0.356156,-1.597548,-1.621256,-0.436408,-1.475613,0.140887,...,-0.233149,-0.396236,-0.10286,-0.058325,-0.090007,-0.172743,-0.206698,-0.187866,-0.351079,-0.815678
Brc1c(Br)c(Br)c2[nH]nnc2c1Br,2.341124,-1.227141,-1.387455,-1.609569,-0.782456,-1.479081,-1.658517,-1.014979,-1.583002,-0.829211,...,-0.233149,-0.396236,-0.10286,-0.058325,-0.090007,-0.172743,-0.206698,-0.187866,-0.351079,-0.237036
Brc1c(NC2=NCCN2)ccc2nccnc12,0.363675,-0.866133,-1.145053,-1.177724,-1.027213,-1.082141,-1.148673,-1.071510,-1.198935,-1.112825,...,-0.233149,-0.396236,-0.10286,-0.058325,-0.090007,-0.172743,-0.206698,-0.187866,-0.351079,1.394061
Brc1cc2c(cc1C1Nc3ccccc3C3C=CCC31)OCO2,-0.430884,-0.363848,-0.711002,-0.677713,-0.527146,-0.551143,-0.520575,-0.456924,-0.438653,-0.369559,...,-0.233149,-0.396236,-0.10286,-0.058325,-0.090007,-0.172743,-0.206698,-0.187866,-0.351079,0.795793
Brc1ccc(-c2csc(N3CCC(c4nn[nH]n4)CC3)n2)cc1,-0.771815,-0.489383,-0.680752,-0.698291,-0.431610,-0.554157,-0.623352,-0.352124,-0.649354,-0.325339,...,-0.233149,-0.396236,-0.10286,-0.058325,11.108842,5.464762,-0.206698,-0.187866,-0.351079,0.911072


Check destination folder

In [20]:
EMBEDDING_DIR

PosixPath('/root/project/src/project_src/state_transition_model/benchmark/chemCPA/chemCPA-main/project_folder/embeddings')

In [21]:
model_name = 'rdkit2D'
dataset_name = 'lincs_trapnell'
fname = f'{model_name}_embedding_{dataset_name}.parquet'

directory = EMBEDDING_DIR /'rdkit' / 'data' /'embeddings'
directory.mkdir(parents=True, exist_ok=True)

Save normalised version

In [22]:
normalized_df.to_parquet(directory / fname)

Check that it worked

In [23]:
df = pd.read_parquet(directory/ fname)
df

,latent_1,latent_2,latent_3,latent_4,latent_5,latent_6,latent_7,latent_8,latent_9,latent_10,...,latent_190,latent_191,latent_192,latent_193,latent_194,latent_195,latent_197,latent_198,latent_199,latent_200
BrC1C(Br)C(Br)C(Br)C(Br)C1Br,1.828414,-1.981056,-1.401597,-1.634367,-0.356156,-1.597548,-1.621256,-0.436408,-1.475613,0.140887,...,-0.233149,-0.396236,-0.10286,-0.058325,-0.090007,-0.172743,-0.206698,-0.187866,-0.351079,-0.815678
Brc1c(Br)c(Br)c2[nH]nnc2c1Br,2.341124,-1.227141,-1.387455,-1.609569,-0.782456,-1.479081,-1.658517,-1.014979,-1.583002,-0.829211,...,-0.233149,-0.396236,-0.10286,-0.058325,-0.090007,-0.172743,-0.206698,-0.187866,-0.351079,-0.237036
Brc1c(NC2=NCCN2)ccc2nccnc12,0.363675,-0.866133,-1.145053,-1.177724,-1.027213,-1.082141,-1.148673,-1.071510,-1.198935,-1.112825,...,-0.233149,-0.396236,-0.10286,-0.058325,-0.090007,-0.172743,-0.206698,-0.187866,-0.351079,1.394061
Brc1cc2c(cc1C1Nc3ccccc3C3C=CCC31)OCO2,-0.430884,-0.363848,-0.711002,-0.677713,-0.527146,-0.551143,-0.520575,-0.456924,-0.438653,-0.369559,...,-0.233149,-0.396236,-0.10286,-0.058325,-0.090007,-0.172743,-0.206698,-0.187866,-0.351079,0.795793
Brc1ccc(-c2csc(N3CCC(c4nn[nH]n4)CC3)n2)cc1,-0.771815,-0.489383,-0.680752,-0.698291,-0.431610,-0.554157,-0.623352,-0.352124,-0.649354,-0.325339,...,-0.233149,-0.396236,-0.10286,-0.058325,11.108842,5.464762,-0.206698,-0.187866,-0.351079,0.911072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c1nc(NC[C@@H]2CCCO2)c2cc(-c3ccc4c(c3)OCO4)ccc2n1,-0.701220,-0.052903,-0.482743,-0.419769,-0.494723,-0.272670,-0.294573,-0.420919,-0.367549,-0.535261,...,-0.233149,-0.396236,-0.10286,-0.058325,-0.090007,-0.172743,-0.206698,-0.187866,-0.351079,1.082694
c1nc(NCc2ccc3c(c2)OCO3)c2cc(-c3ccc4c(c3)OCCO4)ccc2n1,-1.043611,0.711289,-0.100867,-0.084774,-0.159690,0.168823,0.000405,-0.132287,-0.134112,-0.312200,...,-0.233149,-0.396236,-0.10286,-0.058325,-0.090007,-0.172743,-0.206698,-0.187866,-0.351079,-0.099951
c1nc(NCc2ccc3c(c2)OCO3)c2cc(-c3ccoc3)ccc2n1,-0.758711,0.225638,-0.482743,-0.486121,-0.561083,-0.272670,-0.438979,-0.562218,-0.519458,-0.680419,...,-0.233149,-0.396236,-0.10286,-0.058325,-0.090007,-0.172743,-0.206698,-0.187866,-0.351079,0.216967
c1nc2c(NC3CCCCC3)nc(Nc3ccc(N4CCOCC4)cc3)nc2[nH]1,-0.979916,-0.077524,-0.235467,-0.099588,-0.174506,-0.007172,0.100338,-0.034503,-0.021827,-0.204905,...,-0.233149,-0.396236,-0.10286,-0.058325,-0.090007,-0.172743,-0.206698,-0.187866,-0.351079,0.275954
